# Data Processing

This notebook demonstrates how to use `pyTigerGraph` for common data processing tasks on graphs stored in `TigerGraph`.

### Connection to Database

The `TigerGraphConnection` class represents a connection to the TigerGraph database. Under the hood, it stores the necessary information to communicate with the database. It is able to perform quite a few database tasks. Please see its [documentation](https://docs.tigergraph.com/pytigergraph/current/intro/) for details.

**Note**: Secret instead of username/password is required for TG cloud DBs created after 7/5/2022. Otherwise, you can leave it blank.

In [1]:
from pyTigerGraph import TigerGraphConnection

In [2]:
conn = TigerGraphConnection(
    host="http://127.0.0.1", # Change the address to your database server's
    graphname="Cora",
    username="tigergraph",
    password="tigergraph",
    gsqlSecret="" # secret instead of user/pass is required for TG cloud DBs created after 7/5/2022  
)

# Uncomment below to get and set token if token authentication is enabled. 
#conn.getToken(<secret>) # <secret> is your user secret. See https://docs.tigergraph.com/tigergraph-server/current/user-access/managing-credentials#_secrets for details.

In [3]:
# Graph schema and other information.
print(conn.gsql("ls"))

---- Graph Cora
Vertex Types:
- VERTEX Paper(PRIMARY_ID id INT, x LIST<INT>, y INT, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
Edge Types:
- DIRECTED EDGE Cite(FROM Paper, TO Paper, time INT, is_train BOOL, is_val BOOL)

Graphs:
- Graph Cora(Paper:v, Cite:e)
Jobs:
- CREATE LOADING JOB load_cora_data FOR GRAPH Cora {
DEFINE FILENAME edge_csv = "./edges.csv";
DEFINE FILENAME node_csv = "./nodes.csv";
LOAD node_csv TO VERTEX Paper VALUES($"id", SPLIT($"x", " "), $"y", $"train", $"valid", $"test") USING SEPARATOR=",", HEADER="true", EOL="\n";
LOAD edge_csv TO EDGE Cite VALUES($"source", $"target", _, _, _) USING SEPARATOR=",", HEADER="true", EOL="\n";
}

- CREATE SCHEMA_CHANGE JOB Cora_job FOR GRAPH Cora {
ADD VERTEX Paper(PRIMARY_ID id INT, x LIST<INT>, y INT, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true";
ADD DIRECTED EDGE Cite(FROM Paper, TO Paper,

In [4]:
# Number of vertices for every vertex type
conn.getVertexCount('*')

{'Paper': 2708}

In [5]:
# Number of vertices of a specific type
conn.getVertexCount("Paper")

2708

In [6]:
# Number of edges for every type
conn.getEdgeCount()

{'Cite': 10556}

In [7]:
# Number of edges of a specific type
conn.getEdgeCount("Cite")

10556

### Random Vertex Split

The `VertexSplitter` split vertices into at most 3 parts randomly. The split results are stored in the provided vertex attributes. Each boolean attribute indicates which part a vertex belongs to. For example, if you want to split the vertices into 80% train, 10% validation and 10% test, you can provide as arguments to the splitter `train_mask=0.8, val_mask=0.1, test_mask=0.1`. The 3 attributes `train_mask`, `val_mask`, `test_mask` have to exist in the graph. 80% of vertices will be set to `train_mask=True`, 10% to `val_mask=True`, and 10% to `test_mask=True` at random. There will be no overlap between the partitions.

In [8]:
%%time
split = conn.gds.vertexSplitter(train_mask=0.8, val_mask=0.1, test_mask=0.1)


Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 309 ms, sys: 201 ms, total: 509 ms
Wall time: 2min 2s


In [9]:
%%time
split.run()

Splitting vertices...
Vertex split finished successfully.
CPU times: user 4.86 ms, sys: 3.05 ms, total: 7.91 ms
Wall time: 62.8 ms


Now the split is done. Load all vertices and check if the split is correct. See the next tutorial for details on `VertexLoader` and other data loaders. A UDF is needed for the loader. Uncomment below to install the UDF. It only needs to be installed once per query. **Note**: installing the UDF will overwrite any existing UDF in the database.

In [ ]:
ExprFunctions="https://tg-mlworkbench.s3.us-west-1.amazonaws.com/udf/1.0/ExprFunctions.hpp"  # For enterprise users, please use the link you received.
ExprUtil=""  # For enterprise users, please use the link you received.
conn.installUDF(ExprFunctions, ExprUtil)

In [10]:
%%time
vertex_loader = conn.gds.vertexLoader(attributes=["train_mask", "val_mask", "test_mask"])

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 5.47 ms, sys: 3.94 ms, total: 9.41 ms
Wall time: 28.1 s


In [11]:
%%time
data = vertex_loader.data

CPU times: user 19 ms, sys: 1.25 ms, total: 20.2 ms
Wall time: 44.5 ms


In [12]:
data.train_mask.sum()/len(data), data.val_mask.sum()/len(data), data.test_mask.sum()/len(data)

(0.7950516986706057, 0.10118168389955687, 0.10376661742983752)

### Random Edge Split

The `EdgeSplitter` split edges into at most 3 parts randomly. The split results are stored in the provided edge attributes. Each boolean attribute indicates which part an edge belongs to. For example, if you want to split the edges into 80% train and 20% validation, you can provide as arguments to the splitter `is_train=0.8, is_val=0.2`. The 2 attributes `is_train`, `is_val` have to exist in the graph. 80% of edges will be set to `is_train=True`, 20% to `is_val=True` at random. There will be no overlap between the partitions.

In [13]:
%%time
splitter = conn.gds.edgeSplitter(is_train=0.8, is_val=0.2)

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 6.87 ms, sys: 3.78 ms, total: 10.7 ms
Wall time: 28.8 s


In [14]:
%%time
splitter.run()

Splitting edges...
Edge split finished successfully.
CPU times: user 4.56 ms, sys: 247 µs, total: 4.81 ms
Wall time: 82.3 ms


Now the split is done. Load all edges and check if the split is correct. See the next tutorial for details on `EdgeLoader` and other data loaders. A UDF is needed for the loader. Uncomment below to install the UDF. It only needs to be installed once per query. **Note**: installing the UDF will overwrite any existing UDF in the database.

In [ ]:
ExprFunctions="https://tg-mlworkbench.s3.us-west-1.amazonaws.com/udf/1.0/ExprFunctions.hpp"  # For enterprise users, please use the link you received.
ExprUtil=""  # For enterprise users, please use the link you received.
conn.installUDF(ExprFunctions, ExprUtil)

In [15]:
%%time
edge_loader = conn.gds.edgeLoader(attributes=["is_train", "is_val"])

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 7.02 ms, sys: 3.56 ms, total: 10.6 ms
Wall time: 32.8 s


In [16]:
%%time
data = edge_loader.data

CPU times: user 11.3 ms, sys: 1.69 ms, total: 12.9 ms
Wall time: 37.3 ms


In [17]:
data.is_train.sum()/len(data), data.is_val.sum()/len(data)

(0.8020083364910952, 0.19799166350890487)